# 1. Setup Game Libraries

In [ ]:
!pip install gym_super_mario_bros 
!pip install nes_py

In [2]:
# Import the game
import gym_super_mario_bros
# Import the Joypad wrapper
from nes_py.wrappers import JoypadSpace
# Import the SIMPLIFIED controls
from gym_super_mario_bros.actions import SIMPLE_MOVEMENT

# Preprocess Environment

In [ ]:
# Install pytorch (AMD G.P.U Currently not Supported on PyTorch)
!pip3 install torch torchvision torchaudio

In [ ]:
# Install stable baselines for RL stuff
!pip install stable-baselines3[extra]

In [ ]:
# Import Frame Stacker Wrapper and GrayScaling Wrapper
from gym.wrappers import GrayScaleObservation
# Import Vectorization Wrappers
from stable_baselines3.common.vec_env import VecFrameStack, DummyVecEnv
# Import Matplotlib to show the impact of frame stacking
from matplotlib import pyplot as plt

In [ ]:
state = env.reset()
state, reward, done, info = env.step([5])
plt.figure(figsize=(20,16))
for idx in range(state.shape[3]):
    plt.subplot(1,4,idx+1)
    plt.imshow(state[0][:,:,idx])
plt.show()

In [ ]:
# 1. Create the base environment
env = gym_super_mario_bros.make('SuperMarioBros-v0')
# 2. Simplify the controls 
env = JoypadSpace(env, SIMPLE_MOVEMENT)
# 3. Grayscale
env = GrayScaleObservation(env, keep_dim=True)
# 4. Wrap inside the Dummy Environment
env = DummyVecEnv([lambda: env])
# 5. Stack the frames
env = VecFrameStack(env, 4, channels_order='last')

# Train the Reinforcement Learning Model

In [8]:
# Import os for file path management
import os 
# Import PPO for RL Algorithms
from stable_baselines3 import PPO
# Import Base Callback for saving models
from stable_baselines3.common.callbacks import BaseCallback

In [9]:
#Automatic Save Model Function
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'Model_wSteps_{}'.format(self.n_calls))
            self.model.save(model_path)

        return True

In [10]:
#Here models get Saved
CHECKPOINT_DIR = './train/'
LOG_DIR = './logs/'

In [11]:
# Setup model saving callback
#check_freq = Interval of Steps to Save Model
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

In [12]:
# This is the AI model started
#n_steps : steps after which the log is returned
#learning_rate : Rate with which the model learns to play
model = PPO('CnnPolicy', env, verbose=1, tensorboard_log=LOG_DIR, learning_rate=0.000001, 
            n_steps=512) 

Using cpu device
Wrapping the env in a VecTransposeImage.


In [ ]:
# Train the AI model, this is where the AI model starts to learn
# Inshort, this command stars the model learning process
model.learn(total_timesteps=1000000, callback=callback)

# Testing it Out!

In [ ]:
# Loading model
# Comment  LINE 4 this line, if building model for the first time
# Edit LINE 4, where you desire to load the model from
model = PPO.load('./train/Model_wSteps_1000')

In [ ]:
# Start the game 
state = env.reset()
# Loop through the game
while True: 
    action, _ = model.predict(state)
    state, reward, done, info = env.step(action)
    env.render()